In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
#读取数据，处理数据
data = pd.read_csv("D:\桌面\Train.csv")
#把shares大于10000的都删除.占比<5%
rows_to_drop = data[data[' shares'] > 10000].index
df = data.drop(rows_to_drop)
#手动降维
df = df.drop([" n_non_stop_words"," weekday_is_monday"," weekday_is_tuesday"," weekday_is_wednesday"," weekday_is_thursday"," weekday_is_friday"
                ," weekday_is_saturday"," weekday_is_sunday"], axis=1)

In [20]:
# 将训练数据和测试数据分开
X = df.iloc[:, :-1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y = df.iloc[:, -1]

In [21]:
pca = PCA(n_components=15)
X_pca = pca.fit_transform(X_scaled)
X_pca = scaler.fit_transform(X_pca)
# 训练模型并进行十折交叉验证
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=5)
mae_list = []
rmse_list = []
mape_list = []
for train_index, test_index in kf.split(X_pca):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #把y值分为两类来进行预测
    y_binary = np.where(y_train > 1500,1,0)
    lr_model = LogisticRegression(max_iter=200)
    lr_model.fit(X_train, y_binary)
    #把预测值映射回去
    y_pred = lr_model.predict(X_test)
    y_pred= np.where(y_pred > 0, 100*12, 100*8)
    mape_list.append(np.mean(np.abs((y_test - y_pred) / y_test)) * 100)
np.mean(mape_list)

50.848489344578425

In [33]:
sum(y_pred<900)

2295

## 预测数据

In [49]:
test = pd.read_csv('D:\桌面\Test.csv')

In [50]:
test = test.drop([" n_non_stop_words"," weekday_is_monday"," weekday_is_tuesday"," weekday_is_wednesday"," weekday_is_thursday"," weekday_is_friday"
                ," weekday_is_saturday"," weekday_is_sunday"], axis=1)

In [51]:
# 将训练数据和测试数据分开
Xt = test.iloc[:, :-1]
scaler = MinMaxScaler()
Xt = scaler.fit_transform(Xt)

In [52]:
X_pca = pca.transform(Xt)
X_pca = scaler.fit_transform(X_pca)

In [53]:
y = lr_model.predict(X_pca)
y= np.where(y > 0, 100*12, 100*8)

In [54]:
sum(y<900)

1780

In [55]:
ans = pd.read_csv('D:\桌面\Test.csv')

In [56]:
ans[' shares']=y

In [58]:
ans.to_csv('D:\桌面\竞赛1第三次.csv')